In [1]:
from logger import setup_custom_logger
import sys
import pandas as pd    
import cbsodata

logger = setup_custom_logger("B009_New_Building_Permits")
logger.info('starting')
try:
    df = pd.DataFrame(cbsodata.get_data('83671NED',filters="Perioden gt '2012' and Opdrachtgever ne 'T001209' and Eigendom ne'T001258'",select=['RegioS','Perioden','Opdrachtgever','Eigendom', 'Woningen_2', 'Wooneenheden_3','Recreatiewoningen_4']))
except:
    logger.exception('API incorrectly loaded')
    raise

2021-03-18 09:54:56 INFO     B009_New_Building_Permits starting


INFO:B009_New_Building_Permits:starting


In [2]:
try:
    df = df.groupby(['Perioden', 'Opdrachtgever', 'Eigendom']).agg({'Woningen_2': ['sum'], 'Wooneenheden_3': sum, 'Recreatiewoningen_4': sum})
    df = df.reset_index()
except:
    logger.exception('Grouping data failed')
    raise

In [3]:
try:
    df.to_excel("output/B009_New_Building_Permits.xlsx")
except:
    logger.exception('Exporting failed')
    raise

In [4]:
logger.info('Ended')

2021-03-18 09:55:04 INFO     B009_New_Building_Permits Ended


INFO:B009_New_Building_Permits:Ended
